# Handwritten Digit Recognition using k-nearest neighbors
## Lena Fleischhacker, Pia Röhrich, Hellen Röttgen, Benjamin Wehnert
### July 2022

## Abstract

## Table of contents

## Introduction

## Methods

## Results

## Further Applications

## Discussion

## References